In [1]:
import requests
import re
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import deepl
import time
import random
import os
from datetime import datetime
import multiprocessing as mp
from functools import partial

In [2]:
def preserve_speakers_batch(texts):
    """배치로 화자 태그 처리"""
    preserved_texts = []
    placeholder_maps = []
    
    for text in texts:
        if pd.isna(text):
            preserved_texts.append("")
            placeholder_maps.append({})
            continue
            
        speakers = re.findall(r'#Person\d+#', str(text))
        preserved_text = str(text)
        placeholder_map = {}
        
        for i, speaker in enumerate(speakers):
            placeholder = f"SPEAKER_{i}"
            preserved_text = preserved_text.replace(speaker, placeholder)
            placeholder_map[placeholder] = speaker
            
        preserved_texts.append(preserved_text)
        placeholder_maps.append(placeholder_map)
        
    return preserved_texts, placeholder_maps

def restore_speakers_batch(texts, placeholder_maps):
    """배치로 화자 태그 복원"""
    restored_texts = []
    
    for text, placeholder_map in zip(texts, placeholder_maps):
        if not text or not placeholder_map:
            restored_texts.append(text)
            continue
            
        restored_text = str(text)
        for placeholder, speaker in placeholder_map.items():
            restored_text = restored_text.replace(placeholder, speaker)
        restored_texts.append(restored_text)
        
    return restored_texts

def backtranslate_batch(texts, translator):
    """배치 단위로 텍스트 번역"""
    if not texts:
        return texts
        
    try:
        # 한국어 -> 영어
        en_results = translator.translate_text(
            texts,
            source_lang="KO",
            target_lang="EN-US"
        )
        en_texts = [result.text for result in en_results]
        time.sleep(0.1)  # API 제한 방지
        
        # 영어 -> 한국어
        ko_results = translator.translate_text(
            en_texts,
            source_lang="EN",
            target_lang="KO"
        )
        
        return [result.text for result in ko_results]
        
    except Exception as e:
        print(f"Translation error: {str(e)}")
        return texts

def process_chunk(chunk, chunk_id, api_key):
    """청크 단위로 데이터 처리"""
    translator = deepl.Translator(api_key)
    results = []
    
    BATCH_SIZE = 25  # DeepL API의 배치 제한을 고려한 크기
    
    for idx in range(0, len(chunk), BATCH_SIZE):
        batch = chunk[idx:idx + BATCH_SIZE]
        
        # 화자 태그 보존
        preserved_texts, placeholder_maps = preserve_speakers_batch(batch)
        
        # 배치 번역
        translated_texts = backtranslate_batch(preserved_texts, translator)
        
        # 화자 태그 복원
        final_texts = restore_speakers_batch(translated_texts, placeholder_maps)
        
        results.extend(final_texts)
        time.sleep(0.1)  # API 호출 제한 방지
        
    return results

def process_summaries(original_df, save_path, api_key):
    print(f"Translation started: {datetime.now().strftime('%H:%M:%S')}")
    
    # 결과 데이터프레임 준비
    result_df = original_df.copy()
    
    # 멀티프로세싱 설정
    num_cores = mp.cpu_count() - 1
    chunk_size = max(1, len(original_df) // num_cores)
    chunks = [original_df['summary'].tolist()[i:i + chunk_size] 
             for i in range(0, len(original_df), chunk_size)]
    
    with mp.Pool(num_cores) as pool:
        args = [(chunk, i, api_key) for i, chunk in enumerate(chunks)]
        
        results = []
        with tqdm(total=len(chunks), desc="Processing chunks") as pbar:
            for chunk_result in pool.starmap(process_chunk, args):
                results.extend(chunk_result)
                pbar.update(1)
                
                # 중간 저장
                if len(results) % 1000 == 0:
                    temp_df = original_df.copy()
                    temp_df['summary'] = temp_df['summary'].copy()
                    temp_df.loc[:len(results)-1, 'summary'] = results[:len(results)]
                    temp_df.to_csv(f"{save_path}_temp.csv", index=False)
                    print(f"Saved temporary results: {len(results)}/{len(original_df)} processed")
    
    # 최종 데이터프레임 생성
    final_df = pd.DataFrame({
        'fname': original_df['fname'],
        'dialogue': original_df['dialogue'],
        'summary': results,  # 역번역된 summary
        'topic': original_df['topic']
    })
    
    # 최종 결과 저장
    final_df.to_csv(save_path, index=False)
    
    print(f"Completed: {datetime.now().strftime('%H:%M:%S')}")
    print(f"Saved to: {save_path}")
    
    # 샘플 결과 출력
    print("\nSample results (first 3 rows):")
    for i in range(min(3, len(original_df))):
        print(f"\nOriginal: {original_df['summary'].iloc[i]}")
        print(f"Backtranslated: {results[i]}")
    
    return final_df

In [3]:
if __name__ == "__main__":
    # DeepL API 키 설정
    API_KEY = "3e69e331-9c9f-4cf5-811c-8fcd910e36da"  # 여기에 API 키를 입력하세요
    
    # 저장 경로 설정
    save_dir = '/root/NLP/data/final/'
    os.makedirs(os.path.dirname(save_dir), exist_ok=True)
    
    # 타임스탬프를 포함한 파일명 생성
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    save_path = os.path.join(os.path.dirname(save_dir), 
                            f'sum_aug_bt_{timestamp}.csv')
    
    # CSV 파일 읽기
    df = pd.read_csv('/root/NLP/data/final/aug_bt_train.csv')
    
    # 처리 시작
    final_df = process_summaries(df, save_path, API_KEY)

Translation started: 23:41:58


Processing chunks:   0%|          | 0/48 [00:00<?, ?it/s]

Processing chunks:  19%|█▉        | 9/48 [01:49<05:42,  8.79s/it]   

Saved temporary results: 5000/29412 processed


Processing chunks:  35%|███▌      | 17/48 [01:49<01:58,  3.82s/it]

...

Processing chunks:  69%|██████▉   | 33/48 [01:50<00:19,  1.32s/it]

Saved temporary results: 20000/29412 processed


Processing chunks: 100%|██████████| 48/48 [01:50<00:00,  2.30s/it]

Saved temporary results: 25000/29412 processed


Completed: 23:43:49
Saved to: /root/NLP/data/final/sum_aug_bt_20241127_234157.csv

Sample results (first 3 rows):

Original: 스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.
Backtranslated: 스미스 씨는 검진을 받고 있으며, 호킨스 박사는 매년 검진을 받을 것을 권장합니다. 호킨스 박사는 스미스 씨의 금연에 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.

Original: 파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후 리키에게 백신을 접종했다.
Backtranslated: 파커 부인은 리키를 데리고 백신을 맞으러 갔습니다. 피터스 박사는 기록을 확인하고 리키에게 백신을 투여했습니다.

Original: #Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#에게 도움을 청하고 있습니다.
Backtranslated: #Person1#이 키 세트를 찾고 있으며 #Person2#에게 키 찾기를 도와달라고 요청합니다.


In [9]:
dt_df = pd.read_csv("/root/NLP/data/dt_train.csv/backtranslated_20241127_132634.csv")

In [8]:
all_df = pd.concat([df, dt_df],axis=0)

In [9]:
all_df.head()

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스


In [12]:
co_df = pd.read_csv("/root/NLP/data/dt_train.csv/(co)backtranslated_20241127_091942.csv")

In [14]:
co_df.head()

,fname,dialogue,summary,topic
0,train_1000,#Person1#: 거주하시는 도시의 날씨는 어떤가요?\n#Person2#: 여름에...,"#Person2#의 도시에서는 여름에는 덥고 비가 오며, 겨울에는 더 건조하다. 하...",도시 날씨
1,train_1001,"#Person1#: 안녕하세요, 무엇을 도와드릴까요?\n#Person2#: 미국식 ...","#Person1#은 #Person2#가 햇볕에 익힌 계란과 브렉퍼스트 주스, 진한 ...",음식 주문
2,train_1002,"#Person1#: 분명 후회하지 않으실 거라고 확신하며, 그보다 더 좋은 사람은 ...",#Person1#은 #Person2#가 그 여자를 선택하지 않으면 후회할 것이라고 ...,후회
3,train_1003,"#Person1#: 실례합니다만, 저 좀 도와주시겠어요?\n#Person2#: 예....",#Person1#은 #Person2#에게 꽃 전시회에 대해 물어본다. #Person...,꽃 전시회
4,train_1004,#Person1#: 분실물 센터에 있습니다. 어떻게 도와드릴까요?\n#Person2...,#Person2#의 가방이 어제 분실되었다. #Person2#가 제공한 정보를 바탕...,분실물 찾기


In [15]:
all_co_df = pd.concat([all_df, co_df])

In [16]:
#인덱스 리셋
all_co_df.reset_index(drop=True, inplace=True)

# fname 열을 'train_0', 'train_1', ... 으로 변환
all_co_df['fname'] = [f"train_{i}" for i in range(len(all_co_df))]

# 중복 제거 데이터 저장 (원하는 경로로 변경)
output_path = "/root/NLP/data/dt1_train.csv"
all_co_df.to_csv(output_path, index=False)

print("CSV 파일이 성공적으로 저장되었습니다.")

CSV 파일이 성공적으로 저장되었습니다.
